# XGBoost model with Airline dataset. Experiment 01

XGBoost regressor with L2 + Airline dataset

In [ ]:
import os,sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from lightgbm.sklearn import LGBMRegressor
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import (confusion_matrix, accuracy_score, roc_auc_score, f1_score, log_loss, precision_score,
                             recall_score, mean_squared_error, mean_absolute_error, r2_score)
from scipy.stats import gmean

from libs.timer import Timer
from libs.loaders import load_fraud, load_iot, load_airline
from libs.conversion import convert_cols_categorical_to_numeric, convert_related_cols_categorical_to_numeric
from libs.conversion import _get_nominal_integer_dict, _convert_to_integer

print("System version: {}".format(sys.version))

System version: 3.6.0 |Anaconda 4.3.1 (64-bit)| (default, Dec 23 2016, 12:22:00) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]


/home/hoaphumanoid/anaconda3/envs/strata/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Load data

In [ ]:
%%time
df_plane = load_airline()
print(df_plane.shape)


MOUNT_POINT not found in environment. Defaulting to /fileshare


In [ ]:
df_plane.head()

In [ ]:
%%time
df_plane_numeric = convert_related_cols_categorical_to_numeric(df_plane, col_list=['Origin','Dest'])


In [ ]:
df_plane_numeric.head()

In [ ]:
%%time
df_plane_numeric = convert_cols_categorical_to_numeric(df_plane_numeric, col_list='UniqueCarrier')


In [ ]:
df_plane.dtypes

In [ ]:
df_plane_numeric.head()

In [ ]:
%%time
filter_func = lambda x: 1 if x > 0 else 0
df_plane_numeric['ArrDelayBinary'] = df_plane_numeric['ArrDelay'].map(filter_func)

In [ ]:
df_plane_numeric.head()

In [ ]:
%%time
#gets a random 80% of the entire set
X_train = df_plane_numeric.sample(frac=0.8, random_state=1)
print(X_train.shape)
#gets the left out portion of the dataset
X_test = df_plane_numeric.loc[~df_plane_numeric.index.isin(X_train.index)]
print(X_test.shape)

### Training

In [ ]:
# Variables
KFOLD_SPLITS = 5

In [ ]:
def train_xgboost(df, verbose=True):
    if verbose: print("Training data of size {}".format(df.shape))
    x = df[df.columns.difference(['ArrDelay', 'ArrDelayBinary'])].as_matrix()
    y = df['ArrDelayBinary'].as_matrix()
    
    n_splits = 5
    skf = KFold(n_splits=n_splits, random_state=77, shuffle=True)
    clfs = []
    
    if verbose: print("Computing LightGBM boosted tree using {} kfold cross validation".format(n_splits))
    for train_index, test_index in skf.split(x, y):
        trn_x, val_x = x[train_index,:], x[test_index,:]
        trn_y, val_y = y[train_index], y[test_index]

        
        
        clf = xgb.XGBRegressor(max_depth=6,
                            n_estimators=300,
                            min_child_weight=30,
                            learning_rate=0.01,
                            nthread=20,
                            subsample=0.80,
                            colsample_bytree=0.80,
                            seed=77)
        clf.fit(trn_x, trn_y, eval_set=[(val_x, val_y)], verbose=verbose, eval_metric='rmse', early_stopping_rounds=100)
        clfs.append(clf)
        
    return clfs

In [ ]:
%%time
X_train_small = X_train.sample(frac=0.001, random_state=1)
print(X_train_small.shape)

In [ ]:
%%time
#clfs = train_xgboost(X_train_small)
clfs = train_xgboost(X_train)

### Evaluation

In [ ]:
def compute_prediction(clfs, df, verbose=True):    
    x = df[df.columns.difference(['ArrDelay', 'ArrDelayBinary'])].as_matrix()
    if verbose: print(x.shape)
    preds = []
    with Timer() as t:
        for clf in clfs:
            preds.append(np.clip(clf.predict(x), 0.0001, 0.9999))
        pred = gmean(np.array(preds), axis=0)
    if verbose: print("Prediction took %.03f sec.\n" % t.interval) 
    df['prediction'] = pred
    return df

In [ ]:
X_test_small = X_test.sample(frac=0.001, random_state=1)
print(X_test_small.shape)

In [ ]:
%%time
#X_test_pred = compute_prediction(clfs, X_test_small)
X_test_pred = compute_prediction(clfs, X_test)
X_test_pred.head(20)

### Metrics

In [ ]:
#https://github.com/miguelgfierro/codebase/blob/master/python/machine_learning/metrics.py
def classification_metrics_binary(y_true, y_pred):
    m_acc = accuracy_score(y_true, y_pred)
    m_f1 = f1_score(y_true, y_pred)
    m_precision = precision_score(y_true, y_pred)
    m_recall = recall_score(y_true, y_pred)
    m_conf = confusion_matrix(y_true, y_pred)
    report = {'Accuracy':m_acc, 'Precision':m_precision, 'Recall':m_recall, 'F1':m_f1, 'Confusion Matrix':m_conf}
    return report

In [ ]:
#https://github.com/miguelgfierro/codebase/blob/master/python/machine_learning/metrics.py
def classification_metrics_binary_prob(y_true, y_prob):
    m_auc = roc_auc_score(y_true, y_prob)
    m_logloss = log_loss(y_true, y_prob)
    report = {'AUC':m_auc, 'Log loss':m_logloss}
    return report

In [ ]:
THRES = 0.5
threshold_func = lambda x: 0 if x<=THRES else 1


In [ ]:
y_true = X_test_pred['ArrDelayBinary'].as_matrix()
y_prob = X_test_pred['prediction'].as_matrix()
y_pred = X_test_pred['prediction'].map(threshold_func).as_matrix()

In [ ]:
report1 = classification_metrics_binary(y_true, y_pred)
report1

In [ ]:
report2 = classification_metrics_binary_prob(y_true, y_prob)
report2